In [1]:
import sys
print(sys.executable)

sys.path.append('../../')

/home/mshunya/project/100_GQCO/GQCO/.env/bin/python


In [2]:
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from IPython.display import display, Math
from gqco.train import MyModel
from gqco.utils import fix_seed, arange_token
from gqco.data import generate_data
from gqco.solve import solve_from_token, plot_from_dict
from gqco.model import TransformerWithMoE

In [3]:
def data_from_adj(adj, args, num_clone, device):
    dataset = RandomGraphDatasetWithClone(adj, num_clone=num_clone, device=device)
    dataset.x = dataset.x.half()
    dataset.edge_attr = dataset.edge_attr.half()
    record = Batch.from_data_list(dataset)
    record['size'] = record['size'].tolist()[0]
    record['len'] = dataset.len()

    return adj, size, record

In [80]:
task_path = '../../model/taskobjects.pkl'
checkpoint_path = '../../model/merged_model.ckpt'
testdata_path = '../../data/testdata.pkl'
seed=0

with open(task_path, 'rb') as f: 
    obj = pickle.load(f)
taskobj = obj['task']
args = obj['args']

with open(testdata_path, 'rb') as f:
    testdata = pickle.load(f)

In [5]:
model = TransformerWithMoE(args)
model = MyModel.load_from_checkpoint(checkpoint_path, model=model, task=gqco, args=args)
model = model.to('cuda')
device = model.device

In [77]:
size_list = [3,4,5,6,7,8,9,10]

In [82]:
result_cliff = {}
for size in tqdm(size_list):

    count_cliff = 0
    
    with open(f'./outputs/pkl/gqcoans_s42_t2.0_cl100_s{size}.pkl', 'rb') as f:
        dct = pickle.load(f)
    
    for i, (ans, token) in enumerate(zip(dct['answer'], dct['tokens'])):

        while token and token[-1] == 0:
            token.pop()
        
        adj = testdata[size][i]
        qc = taskobj.get_circuit(token, size=len(adj))
        
        if is_clifford_circuit(qc.qc):
            count_cliff += 1

    result_cliff[int(size)] = count_cliff

100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.08it/s]


In [83]:
result_cliff

{3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}